In [ ]:
!pip install -q -U tensorflow-text
!pip install -q tf-models-official

In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [ ]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

train_dir = os.path.join(dataset_dir, 'train')

# remove unused folders to make it easier to load the data
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', label_mode='categorical',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

classNum = int(len(raw_train_ds.class_names))
class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', label_mode='categorical',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/test', label_mode='categorical',
    batch_size=batch_size)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Find in modelList.py
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [ ]:
#print
for text_batch, label_batch in train_ds.take(1):
  for i in range(3):
    print(f'Review: {text_batch.numpy()[i]}')
    label = label_batch.numpy()[i]
    print(f'Label : {label} ({class_names[label]})')

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)

  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)

  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(64, activation='relu')(net)
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(classNum, activation='softmax', name='classifier')(net)

  return tf.keras.Model(text_input, net)

classifier_model = build_classifier_model()
classifier_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [ ]:
#bert_raw_result = classifier_model(tf.constant(text_test))

tf.keras.utils.plot_model(classifier_model)

epochs = 3
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()

num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr, num_train_steps=num_train_steps,
        num_warmup_steps=num_warmup_steps, optimizer_type='adamw')


classifier_model.compile(
    optimizer=optimizer,
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
        metrics=['accuracy'])


print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_ds, validation_data=val_ds,
                          epochs=epochs)

loss, accuracy = classifier_model.evaluate(test_ds)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

dataset_name = 'imdb'
saved_model_path = './{}_bert'.format(dataset_name.replace('/', '_'))
classifier_model.save(saved_model_path, include_optimizer=False)
reloaded_model = tf.saved_model.load(saved_model_path)

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/3
625/625 [==============================] - 150s 234ms/step - loss: 0.4809 - accuracy: 0.7588 - val_loss: 0.3633 - val_accuracy: 0.8370
Epoch 2/3
625/625 [==============================] - 139s 223ms/step - loss: 0.3235 - accuracy: 0.8602 - val_loss: 0.3645 - val_accuracy: 0.8460
Epoch 3/3
782/782 [==============================] - 103s 131ms/step - loss: 0.3734 - accuracy: 0.8550
Loss: 0.3733615279197693
Accuracy: 0.8550400137901306


In [ ]:
def print_my_examples(inputs, results):
  result_for_printing = \
    [f'input: {inputs[i]:<30} : score: {results[i][0]:.6f}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()

In [ ]:
examples = [
    'this is such an amazing movie!',  # this is the same sentence tried earlier
    'The movie was great!',
    'The movie was meh.',
    'The movie was okish.',
    'The movie was terrible...'
    'i don\'t think that is a good show',
    'what a shame that a directer make that film',
    'Actually that is quite decent as a indivual movie'
]

#reloaded_results = tf.sigmoid(reloaded_model(tf.constant(examples)))
# original_results = tf.sigmoid(classifier_model(tf.constant(examples)))
original_results2 = tf.nn.softmax(classifier_model(tf.constant(examples)))

# print('Results from the model in memory:')
# print_my_examples(examples, original_results)

print('Results from the model in memory:')
print_my_examples(examples, original_results2)

#test_loss, test_acc = classifier_model.evaluate(val_ds, verbose=2)

preLabel = ['neg', 'pos']
a = classifier_model.predict( tf.constant(examples) )

#print(a)

for i in a:
  print( preLabel[tf.argmax(i)] , i)



Results from the model in memory:
input: this is such an amazing movie! : score: 0.270227
input: The movie was great!           : score: 0.273231
input: The movie was meh.             : score: 0.369795
input: The movie was okish.           : score: 0.684780
input: The movie was terrible...i don't think that is a good show : score: 0.730501
input: what a shame that a directer make that film : score: 0.697626
input: Actually that is quite decent as a indivual movie : score: 0.367626

pos [0.00326373 0.9967362 ]
pos [0.01085467 0.9891453 ]
pos [0.23345192 0.76654804]
neg [0.88791424 0.11208577]
neg [0.9985832  0.00141687]
neg [0.9180102  0.08198986]
pos [0.22879241 0.77120763]


# References  
https://www.tensorflow.org/text/tutorials/classify_text_with_bert